In [1]:
# import the libraries needed
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Preparing the train and test dataset
---
- We have obtained the dataset from https://www.openpowerlifting.org/.
- For ease of data preprocessing, we have only taken data from the 'SBD' event where contestants are required to do all three exercises, namely Squat, Bench, Deadlift.
- From there, we randomised the data rows in the original CSV file and took out a total of 4,000 data points to be used for our train and test dataset. This dataset was further split into 3,000 for train and 1,000 for test.

In [2]:
train_test_combined = pd.read_csv('../datasets/train_test_combined.csv')
train_test_combined.head()

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
0,Alana Carrasco,F,SBD,Raw,29.5,24-34,24-39,FR-O,68.60,72,...,Yes,USA,AZ,USAPL,IPF,27/8/2017,USA,AZ,NaN,Sun Devil Classic
1,Even Dysjaland,M,SBD,Raw,19.5,20-23,19-23,Juniors 19-23,94.65,105,...,Yes,NaN,NaN,NSF,IPF,13/6/2015,Norway,NaN,Ganddal,Klubbstevne
2,Alessandro Favorito,M,SBD,Single-ply,39.0,35-39,24-39,Open,89.40,90,...,Yes,Italy,NaN,FIPL,IPF,6/11/2004,Italy,NaN,Marina Di Carrara,Coppa Italia A Squadre
3,Sydney Martinez,F,SBD,Single-ply,NaN,NaN,NaN,Girls,55.16,56,...,Yes,USA,NaN,THSWPA,NaN,25/1/2014,USA,TX,JUNIOR HIGH,CARRIZO SPRINGS INVITATIONAL
4,Navy Villar,F,SBD,Single-ply,NaN,NaN,NaN,Girls,55.61,56,...,Yes,NaN,NaN,THSWPA,NaN,18/1/2018,USA,TX,Sadler,S&S Lady Rams Meet


In [3]:
# Split combined dataset into train and test
from sklearn.model_selection import train_test_split

train,test = train_test_split(train_test_combined, test_size = 0.25, random_state = 0)
train.to_csv('../datasets/train.csv', index = False)
test.to_csv('../datasets/test.csv', index = False)

print(train.shape)
print(test.shape)

(3000, 41)
(1000, 41)


In [4]:
train_df = pd.read_csv('../datasets/train.csv')

# Removing disqualified lifters
---

Information in the `Place` column allows us to know if certain lifters were disqualified from the competition.

Values are as follows:

- Positive number: the place the lifter came in.
- G: Guest lifter. The lifter succeeded, but wasn't eligible for awards.
- DQ: Disqualified. Note that DQ could be for procedural reasons, not just failed attempts.
- DD: Doping Disqualification. The lifter failed a drug test.
- NS: No-Show. The lifter did not show up on the meet day.


We would want to remove disqualified lifters, as they may negatively impact the outcome of the predictions.

In [5]:
train_df['Place'].unique()

array(['1', '3', 'DQ', '7', '9', '2', '4', '6', 'NS', '5', '14', '11',
       '10', '8', '13', '12', '18', '15', '22', '24', '32', '26', '17',
       '19', '16', 'G', '28', 'DD', '23', '51', '21', '84', '20', '29'],
      dtype=object)

In [6]:
df = train_df.drop(train_df[
    (train_df.Place == 'DQ') |
    (train_df.Place == 'NS') |
    (train_df.Place == 'DD')
].index)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2814 entries, 0 to 2999
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              2814 non-null   object 
 1   Sex               2814 non-null   object 
 2   Event             2814 non-null   object 
 3   Equipment         2814 non-null   object 
 4   Age               1613 non-null   float64
 5   AgeClass          1884 non-null   object 
 6   BirthYearClass    1721 non-null   object 
 7   Division          2812 non-null   object 
 8   BodyweightKg      2775 non-null   float64
 9   WeightClassKg     2788 non-null   object 
 10  Squat1Kg          1284 non-null   float64
 11  Squat2Kg          1270 non-null   float64
 12  Squat3Kg          1230 non-null   float64
 13  Squat4Kg          11 non-null     float64
 14  Best3SquatKg      2787 non-null   float64
 15  Bench1Kg          1285 non-null   float64
 16  Bench2Kg          1277 non-null   float64


After dropping disqualified and no-show lifters from the dataset, we are left with only 2814 values from the original 3000 values in the train dataset. 

# Filling Missing `Age` Values
---

## Using `AgeClass`

First, use the `AgeClass` column to fill in missing values for `Age` column. 

There are a variety of possible AgeClasses that a lifter can fall into, described in https://gitlab.com/openpowerlifting/opl-data/blob/main/modules/opltypes/src/ageclass.rs 

There is an AgeClass named 5-Dec, which does not provide much information about the ages of the lifters. 

However, the `5-Dec` class has only one lifter with a missing `Age` value. We can use the median age of those in the `5-Dec` class to fill this one missing value.

In [8]:
df['AgeClass'].unique()

array(['60-64', nan, '20-23', '55-59', '13-15', '16-17', '24-34', '45-49',
       '70-74', '18-19', '40-44', '35-39', '50-54', '5-Dec', '65-69',
       '80-999', '75-79'], dtype=object)

In [9]:
## 5-Dec AgeClass, with only one missing Age Value
df[df.AgeClass == '5-Dec']

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
307,Davis Lee,M,SBD,Raw,11.0,5-Dec,NaN,Youth 10-11,65.54,67.5,...,Yes,USA,PA,ADAU,NaN,11/11/2012,USA,PA,Bigler,Central PA Open
327,Brady Dibble,M,SBD,Raw,9.5,5-Dec,NaN,MR-Y2,36.60,40,...,Yes,USA,WY,USAPL,IPF,2/2/2019,USA,LA,NaN,Youth Nationals
350,Addyson Navarro,F,SBD,Raw,10.0,5-Dec,NaN,FR-Y2,34.80,35,...,Yes,USA,AZ,USAPL,IPF,23/9/2018,USA,AZ,NaN,Apeman Strong Fest 2.0
411,Casey Walker #1,M,SBD,Raw,10.0,5-Dec,NaN,Youth 11 & Under,41.96,48,...,Yes,USA,PA,RAW,NaN,7/12/2014,USA,PA,Clearfield,Central PA Open
1499,T. McIntosh,M,SBD,Single-ply,NaN,5-Dec,NaN,Open,105.00,105,...,NaN,NaN,NaN,USPF,IPF,23/4/1983,USA,IN,West Lafayette,Indiana State Natural
1608,Ferrell,M,SBD,Wraps,11.5,5-Dec,NaN,Nov-13,82.50,82.5,...,Yes,NaN,NaN,WNPF,NaN,17/6/2000,USA,MD,Baltimore,Teen Junior Submaster & Master Nationals & Ame...
1683,Brian Foster,M,SBD,Single-ply,4.5,5-Dec,NaN,Open,90.00,90,...,Yes,NaN,NaN,ADFPA,NaN,20/11/1983,USA,NJ,West Orange,New Jersey State Drug Free
1928,Hannah Johnson #3,F,SBD,Raw,11.5,5-Dec,NaN,FR-Y3,57.56,63,...,Yes,USA,LA,USAPL,IPF,1/12/2018,USA,LA,NaN,Shreveport Showdown
2215,Rayder Knaack,M,SBD,Raw,10.5,5-Dec,NaN,MR-Y2,46.75,48,...,Yes,USA,OH,USAPL,NaN,15/1/2022,USA,OH,NaN,New Year Open


In [10]:
## find the median age of the 5-Dec AgeClass
df[df.AgeClass == '5-Dec']['Age'].median()

10.25

In [11]:
df.loc[df.AgeClass=='5-Dec', 'Age'] = df.loc[df.AgeClass=='5-Dec', 'Age'].fillna(10.5)
## StackOverflow magic https://stackoverflow.com/questions/49963491/fillna-by-selected-rows-in-pandas-dataframe 


In [12]:
df[df.AgeClass == '5-Dec']

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
307,Davis Lee,M,SBD,Raw,11.0,5-Dec,NaN,Youth 10-11,65.54,67.5,...,Yes,USA,PA,ADAU,NaN,11/11/2012,USA,PA,Bigler,Central PA Open
327,Brady Dibble,M,SBD,Raw,9.5,5-Dec,NaN,MR-Y2,36.60,40,...,Yes,USA,WY,USAPL,IPF,2/2/2019,USA,LA,NaN,Youth Nationals
350,Addyson Navarro,F,SBD,Raw,10.0,5-Dec,NaN,FR-Y2,34.80,35,...,Yes,USA,AZ,USAPL,IPF,23/9/2018,USA,AZ,NaN,Apeman Strong Fest 2.0
411,Casey Walker #1,M,SBD,Raw,10.0,5-Dec,NaN,Youth 11 & Under,41.96,48,...,Yes,USA,PA,RAW,NaN,7/12/2014,USA,PA,Clearfield,Central PA Open
1499,T. McIntosh,M,SBD,Single-ply,10.5,5-Dec,NaN,Open,105.00,105,...,NaN,NaN,NaN,USPF,IPF,23/4/1983,USA,IN,West Lafayette,Indiana State Natural
1608,Ferrell,M,SBD,Wraps,11.5,5-Dec,NaN,Nov-13,82.50,82.5,...,Yes,NaN,NaN,WNPF,NaN,17/6/2000,USA,MD,Baltimore,Teen Junior Submaster & Master Nationals & Ame...
1683,Brian Foster,M,SBD,Single-ply,4.5,5-Dec,NaN,Open,90.00,90,...,Yes,NaN,NaN,ADFPA,NaN,20/11/1983,USA,NJ,West Orange,New Jersey State Drug Free
1928,Hannah Johnson #3,F,SBD,Raw,11.5,5-Dec,NaN,FR-Y3,57.56,63,...,Yes,USA,LA,USAPL,IPF,1/12/2018,USA,LA,NaN,Shreveport Showdown
2215,Rayder Knaack,M,SBD,Raw,10.5,5-Dec,NaN,MR-Y2,46.75,48,...,Yes,USA,OH,USAPL,NaN,15/1/2022,USA,OH,NaN,New Year Open


The `Age` value has been successfully filled. We used a value of 10.5 as required by the definition of the `Age` feature in the documentation.

https://openpowerlifting.gitlab.io/opl-csv/bulk-csv-docs.html


Other points to note about the `AgeClass` column:

-   The largest class consists of lifters 80 years old and above, but the class is named `80-999`. 
-   The smallest class consists of lifters aged `5-12`.  

For simplicity, we will assume that the lifter's age is the midpoint of the boundary defined by the AgeClass. We will use string slicing to extract this middle age (last two characters of the `AgeClass` string). 

We will assume that the maximum age for the `80-999` class is 99 years.

In [13]:
df[df['Age'].isna() & df['AgeClass'].notna()]

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
27,B. Hill,M,SBD,Single-ply,NaN,24-34,24-39,Novice,56.00,56,...,NaN,NaN,NaN,USPF,IPF,5/4/1980,USA,TN,Chattanooga,Southeastern Open & Novice
31,Monique Avalos,F,SBD,Single-ply,NaN,16-17,NaN,Girls,55.34,56,...,Yes,NaN,NaN,THSWPA,NaN,20/2/2016,USA,TX,Seguin Texas,Seguin Matador Meet
43,Acasia Martinez,F,SBD,Single-ply,NaN,16-17,NaN,Girls,55.16,56,...,Yes,NaN,NaN,THSWPA,NaN,18/2/2017,USA,TX,Altair Rice,Rice Consolidated Saturday Meet
61,Jamie Comer,F,SBD,Single-ply,NaN,18-19,NaN,Girls,85.18,90,...,Yes,NaN,NaN,THSWPA,NaN,17/2/2018,USA,TX,Normangee,Normangee Powerlifting Meet
74,Brooke Daniel,F,SBD,Single-ply,NaN,18-19,NaN,Girls,51.26,51.9,...,Yes,NaN,NaN,THSWPA,NaN,26/2/2019,USA,TX,GHS Indoor,Region 6 Div 2 Championship
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2954,Ruby Hernandez,F,SBD,Single-ply,NaN,16-17,14-18,Girls,95.53,100,...,Yes,NaN,NaN,THSWPA,NaN,3/3/2015,USA,TX,Gatesville,Reg 2 Div 3 Championship
2955,Kayla Anderson #3,F,SBD,Single-ply,NaN,18-19,NaN,Girls,93.71,100,...,Yes,NaN,NaN,THSWPA,NaN,24/1/2019,USA,TX,Henderson,Lion Invitational
2977,Thomas Waal,M,SBD,Raw,NaN,24-34,NaN,Open,82.15,83,...,Yes,NaN,NaN,NSF,IPF,16/12/2017,Norway,NaN,Skien,Åpent stevne
2980,Miranda Ovalle,F,SBD,Single-ply,NaN,18-19,NaN,Girls,66.98,67.3,...,Yes,NaN,NaN,THSWPA,NaN,14/2/2019,USA,TX,TSS Gym,TSS-UTSA Invitational 6


In [23]:
def newString(g):
    ## Function that returns the midpoint of an AgeClass 
    if type(g) == str:
        low = high = ""
        for x in range(len(g)):
            if g[x] == '-':
                high += g[x+1]
                high += g[x+2]
                break
            low += g[x]
        return (float(low)+float(high)) / 2
    return g

In [18]:
## First, fill the values using the AgeClass Strings
df['Age'].fillna(df.AgeClass, inplace=True)

In [19]:
## Use the newString function to replace the Strings with the maximum age 
df['Age'] = df['Age'].apply(newString)

In [22]:
## cell for checking
df.loc[2981]

Name                      J. Ashworth
Sex                                 M
Event                             SBD
Equipment                  Single-ply
Age                              37.0
AgeClass                        35-39
BirthYearClass                    NaN
Division             Submasters 35-39
BodyweightKg                      NaN
WeightClassKg                    125+
Squat1Kg                          NaN
Squat2Kg                          NaN
Squat3Kg                          NaN
Squat4Kg                          NaN
Best3SquatKg                    237.5
Bench1Kg                          NaN
Bench2Kg                          NaN
Bench3Kg                          NaN
Bench4Kg                          NaN
Best3BenchKg                    145.0
Deadlift1Kg                       NaN
Deadlift2Kg                       NaN
Deadlift3Kg                       NaN
Deadlift4Kg                       NaN
Best3DeadliftKg                 242.5
TotalKg                         625.0
Place       

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2814 entries, 0 to 2999
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              2814 non-null   object 
 1   Sex               2814 non-null   object 
 2   Event             2814 non-null   object 
 3   Equipment         2814 non-null   object 
 4   Age               1884 non-null   float64
 5   AgeClass          1884 non-null   object 
 6   BirthYearClass    1721 non-null   object 
 7   Division          2812 non-null   object 
 8   BodyweightKg      2775 non-null   float64
 9   WeightClassKg     2788 non-null   object 
 10  Squat1Kg          1284 non-null   float64
 11  Squat2Kg          1270 non-null   float64
 12  Squat3Kg          1230 non-null   float64
 13  Squat4Kg          11 non-null     float64
 14  Best3SquatKg      2787 non-null   float64
 15  Bench1Kg          1285 non-null   float64
 16  Bench2Kg          1277 non-null   float64


In [99]:
## cell for checking if Age was filled correctly
df.loc[2981]

Name                      J. Ashworth
Sex                                 M
Event                             SBD
Equipment                  Single-ply
Age                              39.0
AgeClass                        35-39
BirthYearClass                    NaN
Division             Submasters 35-39
BodyweightKg                      NaN
WeightClassKg                    125+
Squat1Kg                          NaN
Squat2Kg                          NaN
Squat3Kg                          NaN
Squat4Kg                          NaN
Best3SquatKg                    237.5
Bench1Kg                          NaN
Bench2Kg                          NaN
Bench3Kg                          NaN
Bench4Kg                          NaN
Best3BenchKg                    145.0
Deadlift1Kg                       NaN
Deadlift2Kg                       NaN
Deadlift3Kg                       NaN
Deadlift4Kg                       NaN
Best3DeadliftKg                 242.5
TotalKg                         625.0
Place       

## Using `BirthYearClass`


Now the age column only contains numeric float64 datatypes, which shows that we have successfully replaced all the strings after the fillna() operation.

However, there are still some missing values, where both `Age` and `AgeClass` data was missing.

We can fill the remaining missing values for `Age` using the `BirthYearClass` information.


Information about `BirthYearClass`: 

     The ages in the range are the oldest possible ages for the lifter that year. For example, 40-49 means "the year the lifter turns 40 through the full year in which the lifter turns 49."

     BirthYearClass is used primarily by the IPF and by IPF affiliates. Non-IPF federations tend to use AgeClass instead.

     The full range available to BirthYearClass is defined by modules/opltypes/src/birthyearclass.rs. 

We will also use the midpoint of the range defined in `BirthYearClass` to fill out the missing `Age` values. This method is similar to the approach used with `AgeClass`.

In [124]:
df[df['Age'].isna() & df['BirthYearClass'].notna()]

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
27,B. Hill,M,SBD,Single-ply,NaN,24-34,24-39,Novice,56.00,56,...,NaN,NaN,NaN,USPF,IPF,5/4/1980,USA,TN,Chattanooga,Southeastern Open & Novice
40,Ioan Ordean,M,SBD,Single-ply,NaN,NaN,19-23,Junior I,122.50,120+,...,Yes,NaN,NaN,FRPL,IPF,1/6/2013,Romania,NaN,NaN,Campionatul National de Powerlifting Juniori s...
54,John Mitchell #7,M,SBD,Raw,NaN,NaN,50-59,M-C-M2,99.50,100,...,Yes,England,WM,BP,IPF,4/9/2010,England,NaN,Birmingham,British Unequipped Championship
137,D. Mullins,M,SBD,Single-ply,NaN,40-44,40-49,Masters 40-44,125.00,125,...,NaN,NaN,NaN,USPF,IPF,13/5/1989,USA,TX,Dallas,Masters Nationals
183,Glenn France,M,SBD,Single-ply,NaN,16-17,14-18,M-T2,65.00,67.5,...,Yes,USA,NaN,USAPL,IPF,22/11/2003,USA,TX,Austin,Longhorn Open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2867,Henry Francisco,M,SBD,Raw,NaN,20-23,19-23,Juniors,77.80,83,...,Yes,Canada,SK,CPU,IPF,21/5/2016,Canada,SK,Regina,War Of The West
2889,Taylor Lewis #1,F,SBD,Single-ply,NaN,16-17,14-18,Girls,53.98,56,...,Yes,NaN,NaN,THSWPA,NaN,13/2/2019,USA,TX,Baytown,GCM LQM Girls
2912,Nathan Crews,M,SBD,Wraps,NaN,NaN,24-39,Submasters,98.07,100,...,NaN,NaN,NaN,SPF,NaN,9/12/2017,USA,TN,Cleveland,Gritmas Classic
2954,Ruby Hernandez,F,SBD,Single-ply,NaN,16-17,14-18,Girls,95.53,100,...,Yes,NaN,NaN,THSWPA,NaN,3/3/2015,USA,TX,Gatesville,Reg 2 Div 3 Championship


In [125]:
df['BirthYearClass'].unique()

array(['60-69', nan, '50-59', '14-18', '24-39', '40-49', '70-999',
       '19-23'], dtype=object)

In [24]:
## First, fill the values using the AgeClass Strings
df['Age'].fillna(df.BirthYearClass, inplace=True)

In [25]:
## Use the newString function to replace the Strings with the maximum age 
df['Age'] = df['Age'].apply(newString)

In [26]:
## cell for checking
df.loc[2993]

Name                Lily Riggs
Sex                          F
Event                      SBD
Equipment                  Raw
Age                       21.0
AgeClass                   NaN
BirthYearClass           19-23
Division               Juniors
BodyweightKg             69.15
WeightClassKg               72
Squat1Kg                 135.0
Squat2Kg                 142.5
Squat3Kg                -145.0
Squat4Kg                   NaN
Best3SquatKg             142.5
Bench1Kg                  72.5
Bench2Kg                 -77.5
Bench3Kg                  77.5
Bench4Kg                   NaN
Best3BenchKg              77.5
Deadlift1Kg              160.0
Deadlift2Kg              165.0
Deadlift3Kg              170.0
Deadlift4Kg                NaN
Best3DeadliftKg          170.0
TotalKg                  390.0
Place                        2
Dots                    397.11
Wilks                   391.29
Glossbrenner            345.01
Goodlift                  80.7
Tested                     Yes
Country 

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2814 entries, 0 to 2999
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              2814 non-null   object 
 1   Sex               2814 non-null   object 
 2   Event             2814 non-null   object 
 3   Equipment         2814 non-null   object 
 4   Age               1941 non-null   float64
 5   AgeClass          1884 non-null   object 
 6   BirthYearClass    1721 non-null   object 
 7   Division          2812 non-null   object 
 8   BodyweightKg      2775 non-null   float64
 9   WeightClassKg     2788 non-null   object 
 10  Squat1Kg          1284 non-null   float64
 11  Squat2Kg          1270 non-null   float64
 12  Squat3Kg          1230 non-null   float64
 13  Squat4Kg          11 non-null     float64
 14  Best3SquatKg      2787 non-null   float64
 15  Bench1Kg          1285 non-null   float64
 16  Bench2Kg          1277 non-null   float64


In [34]:
df.groupby('WeightClassKg').mean()

,Age,BodyweightKg,Squat1Kg,Squat2Kg,Squat3Kg,Squat4Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,...,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Deadlift4Kg,Best3DeadliftKg,TotalKg,Dots,Wilks,Glossbrenner,Goodlift
WeightClassKg,,,,,,,,,,,,,,,,,,,,,
100,31.274074,97.748100,166.046267,135.497838,74.515634,-225.0,224.016784,118.674800,64.543919,-22.942222,...,179.222667,176.997671,44.825652,-271.0,229.478090,598.372250,382.683850,377.573950,357.283500,69.820700
100+,17.500000,112.214545,NaN,NaN,NaN,NaN,143.070833,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,141.370000,359.095000,271.666364,267.901818,228.167273,47.520909
103,33.250000,99.995000,NaN,NaN,NaN,NaN,241.250000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,256.250000,671.250000,413.400000,408.775000,390.485000,70.235000
105,30.870370,101.130612,164.765333,162.131081,63.778571,NaN,218.190816,124.176000,89.671233,-1.591549,...,209.556000,169.160811,37.471831,NaN,244.185567,605.963265,371.227653,367.257755,350.783776,73.168265
108.2,NaN,108.200000,NaN,NaN,NaN,NaN,265.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,290.000000,732.500000,436.570000,433.310000,413.960000,73.870000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,NaN,86.050000,195.040000,199.580000,204.120000,NaN,204.120000,120.200000,124.740000,-131.540000,...,224.530000,231.330000,235.870000,NaN,235.870000,564.720000,373.760000,369.220000,354.580000,63.880000
90,30.274510,88.286026,139.825692,102.203284,35.634262,142.5,193.542212,107.792239,74.148657,-12.120313,...,190.823731,144.695970,2.805410,97.5,208.726947,525.607904,362.156376,356.303974,335.720568,65.032795
90+,32.333333,104.674444,97.142857,51.785714,108.571429,NaN,149.351429,73.214286,32.142857,34.642857,...,164.642857,51.785714,-58.214286,NaN,166.557857,400.899286,338.745556,332.595556,284.898889,69.704444


In [102]:
t = df.groupby('Division')['Age'].median()
t ## Series showing median age of each division

Division
13-16      15.5
17-19      19.0
18 & 19    18.5
18-19      18.5
40-49       NaN
           ... 
wnov       31.5
wopen      32.5
wpure       NaN
wsm2       39.0
wteen      17.0
Name: Age, Length: 382, dtype: float64

In [103]:
df['Age'] = df['Age'].fillna(df.groupby('Division')['Age'].transform('median'))

##stackoverflow magic https://stackoverflow.com/questions/19966018/pandas-filling-missing-values-by-mean-in-each-group


In [106]:
## cell for checking. If the numbers are the same, the values have been filled correctly
print(df.loc[3]['Age']) ## value in the Age column after the fillna()
print(t[df.loc[3]['Division']]) ## median value of the division

17.75
17.75


,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
40,Ioan Ordean,M,SBD,Single-ply,NaN,NaN,19-23,Junior I,122.5,120+,...,Yes,NaN,NaN,FRPL,IPF,1/6/2013,Romania,NaN,NaN,Campionatul National de Powerlifting Juniori s...
1014,Al Statman,M,SBD,Single-ply,NaN,NaN,60-69,M-M3,90.0,90,...,Yes,USA,NaN,USAPL,IPF,15/5/2004,USA,VA,Oakton,Nations Capital Cup
1117,Adam Anderson #1,M,SBD,Raw,NaN,NaN,40-49,M-C-M1,102.7,105,...,Yes,England,NaN,BP,IPF,20/10/2018,UK,NaN,Belfast,British Classic Masters Powerlifting Champions...
1655,Jeff Dispensa,M,SBD,Wraps,NaN,NaN,50-59,50-59,90.0,90,...,Yes,NaN,NaN,WNPF,NaN,14/12/2014,USA,NJ,Bordentown,Serge McCray Powerlifting & Single Lift Champi...
2781,Chris Baker #1,M,SBD,Raw,NaN,NaN,40-49,M-C-M1,140.0,120+,...,Yes,England,YNE,EPA,IPF,17/9/2017,England,NaN,NaN,YNE Masters


From the documentation (https://openpowerlifting.gitlab.io/opl-csv/bulk-csv-docs.html): 

          Ages can be one of two types: exact or approximate. Exact ages are given as integer numbers, for example 23. Approximate ages are given as an integer plus 0.5, for example 23.5.
     
          A lifter with the given age 23.5 could be either 23 or 24 -- we don't have enough information to know.

Given that the median values we have used to fill the data could be out of the defined values for age (e.g. 17.75), it would help for us to round off all the Age values to the nearest whole number

In [107]:
df['Age'] = df['Age'].apply(lambda x: round(x, 0))

In [110]:
df['Age'].unique()

array([61., 28., 18., 23., 58., 14., 16., 25., 34., 46., 73., 21., 41.,
       30., 17., 27., nan, 22., 45., 39., 20., 54., 19., 32., 33., 38.,
       40., 36., 24., 37., 29., 44., 26., 60., 50., 31., 42., 59., 64.,
       57., 49., 15., 52., 43., 56., 11., 53., 10., 51., 55., 62., 12.,
       35., 68., 13., 65., 48., 72., 66., 80., 70., 47., 75.,  4., 69.,
       71., 74.])

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2814 entries, 0 to 2999
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              2814 non-null   object 
 1   Sex               2814 non-null   object 
 2   Event             2814 non-null   object 
 3   Equipment         2814 non-null   object 
 4   Age               2752 non-null   float64
 5   AgeClass          1884 non-null   object 
 6   BirthYearClass    1721 non-null   object 
 7   Division          2812 non-null   object 
 8   BodyweightKg      2775 non-null   float64
 9   WeightClassKg     2788 non-null   object 
 10  Squat1Kg          1284 non-null   float64
 11  Squat2Kg          1270 non-null   float64
 12  Squat3Kg          1230 non-null   float64
 13  Squat4Kg          11 non-null     float64
 14  Best3SquatKg      2787 non-null   float64
 15  Bench1Kg          1285 non-null   float64
 16  Bench2Kg          1277 non-null   float64


Drop the remaining values NA values for age

In [ ]:
df.dropna(subset=['Age'], inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2793 entries, 0 to 2999
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              2793 non-null   object 
 1   Sex               2793 non-null   object 
 2   Event             2793 non-null   object 
 3   Equipment         2793 non-null   object 
 4   Age               2793 non-null   float64
 5   AgeClass          2228 non-null   object 
 6   BirthYearClass    2114 non-null   object 
 7   Division          2793 non-null   object 
 8   BodyweightKg      2790 non-null   float64
 9   WeightClassKg     2780 non-null   object 
 10  Squat1Kg          1074 non-null   float64
 11  Squat2Kg          1066 non-null   float64
 12  Squat3Kg          1044 non-null   float64
 13  Squat4Kg          4 non-null      float64
 14  Best3SquatKg      1850 non-null   float64
 15  Bench1Kg          1696 non-null   float64
 16  Bench2Kg          1687 non-null   float64


# Filling Missing Values (Bodyweight)

In [ ]:
df[df.BodyweightKg.isna()]

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
1086,Richard Nowazek,M,SBD,Single-ply,51.5,50-54,50-59,Open,NaN,140+,...,Yes,Canada,BC,CPU,IPF,6/4/2002,Canada,BC,Vancouver,BC Provincial Championships
1597,Madison Neesmith,F,B,Raw,20.0,20-23,19-23,JR,NaN,82.5+,...,Yes,USA,NC,WNPF,NaN,6/10/2018,USA,NaN,NaN,World Tournament Of Championships
1605,Jay Bakke,M,SBD,Multi-ply,53.0,50-54,50-59,M_MEM_3_AAPF,NaN,NaN,...,Yes,USA,NaN,APF,WPC,17/7/2010,USA,MT,NaN,Big Sky State Games


We can use the weightclasses to fill in the first 2 missing bodyweight values. We assume they hold the minimum weight in the particular weightclass

Weight classes can be specified as a maximum or as a minimum. Maximums are specified by just the number, for example `90` means "up to (and including) 90kg." minimums are specified by a `+` to the right of the number, for example `90+` means "above (and excluding) 90kg.

In [ ]:
df['BodyweightKg'].fillna(df.WeightClassKg, inplace=True)

In [ ]:
df['BodyweightKg'] = df['BodyweightKg'].apply(lambda f: float(f[:-1]) if type(f) == str else f)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2793 entries, 0 to 2999
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              2793 non-null   object 
 1   Sex               2793 non-null   object 
 2   Event             2793 non-null   object 
 3   Equipment         2793 non-null   object 
 4   Age               2793 non-null   float64
 5   AgeClass          2228 non-null   object 
 6   BirthYearClass    2114 non-null   object 
 7   Division          2793 non-null   object 
 8   BodyweightKg      2792 non-null   float64
 9   WeightClassKg     2780 non-null   object 
 10  Squat1Kg          1074 non-null   float64
 11  Squat2Kg          1066 non-null   float64
 12  Squat3Kg          1044 non-null   float64
 13  Squat4Kg          4 non-null      float64
 14  Best3SquatKg      1850 non-null   float64
 15  Bench1Kg          1696 non-null   float64
 16  Bench2Kg          1687 non-null   float64


The final missing bodyweight value belongs to a Male in the 50-54 AgeClass. We can use the median value of that AgeClass to fill the remaining missing value.

In [ ]:
df.groupby(['AgeClass']).median()

,Age,BodyweightKg,Squat1Kg,Squat2Kg,Squat3Kg,Squat4Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,...,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Deadlift4Kg,Best3DeadliftKg,TotalKg,Dots,Wilks,Glossbrenner,Goodlift
AgeClass,,,,,,,,,,,,,,,,,,,,,
13-15,14.50,58.700,98.75,96.50,83.75,65.0,104.330,52.500,57.50,37.500,...,108.750,118.75,101.05,-105.00,115.00,220.00,211.270,213.220,208.210,47.030
16-17,16.50,70.220,115.00,122.50,96.50,NaN,126.005,70.000,62.50,-35.875,...,128.750,138.75,130.00,-226.25,137.50,300.00,274.870,273.400,256.490,53.130
18-19,18.50,73.900,137.50,147.50,112.50,NaN,142.500,87.500,90.00,-60.000,...,170.000,173.75,125.00,-125.00,154.22,331.12,306.720,302.640,280.990,59.310
20-23,21.50,81.200,170.00,170.00,118.75,165.0,186.250,105.000,107.50,56.250,...,185.000,190.00,130.00,327.50,212.50,422.50,337.770,334.795,315.670,71.550
24-34,28.00,83.000,147.50,157.50,127.50,241.0,182.500,120.000,115.00,-55.000,...,190.000,190.00,126.25,125.00,210.00,320.00,299.400,298.430,270.620,73.910
35-39,37.00,86.910,140.00,117.50,117.50,NaN,193.750,119.000,90.00,-52.500,...,177.500,156.25,142.50,137.50,210.00,267.50,181.200,181.590,173.640,71.165
40-44,42.00,91.900,165.00,158.76,117.50,NaN,190.000,132.500,117.50,-67.500,...,200.000,203.75,120.00,NaN,218.75,240.00,151.900,154.030,146.390,70.120
45-49,47.00,89.050,123.75,151.25,132.50,NaN,182.500,130.000,105.00,-50.000,...,150.000,160.00,157.50,188.00,192.64,225.00,160.365,158.555,142.755,67.210
5-Dec,11.00,39.900,30.00,45.00,-38.75,NaN,25.000,23.750,20.00,-22.500,...,42.215,63.50,-40.00,-86.18,35.00,35.00,44.490,46.740,46.350,21.615


In [ ]:
df.BodyweightKg.fillna(82.45, inplace=True)

In [1]:
print('End of file ')

End of file 
